In [1]:
#Set up your environment:
#pip install transformers torch

In [2]:
from transformers import BertTokenizer, BertForTokenClassification, BertTokenizerFast, AutoTokenizer
from transformers import pipeline
import pandas as pd
import pickle
from tqdm import tqdm
import torch
import spacy
import tensorflow as tf
import numpy as np

In [3]:
# Load the pre-trained model and tokenizer
model_name = "bert-base-uncased"
num_labels = 17
tokenizer = BertTokenizerFast.from_pretrained(model_name)
#model = BertForTokenClassification.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(
    model_name,
    num_labels=41,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
df = pd.read_csv("ner_datasetreference.csv",encoding='latin-1')
df = df.ffill(axis = 0)
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [5]:
df['POS'].nunique()

42

In [6]:
# Creating the sentences and the entities in the below format
#("Apple is looking to buy a startup called XYZ Corp in New York.", {"entities": [(0, 5, "ORG"), (35, 42, "ORG"), (46, 54, "GPE")]}),
unique_sent_num =  df['Sentence #'].unique()
unique_sent_num

array(['Sentence: 1', 'Sentence: 2', 'Sentence: 3', ...,
       'Sentence: 47957', 'Sentence: 47958', 'Sentence: 47959'],
      dtype=object)

In [7]:
def sentence_entities(id):
    tempdf = df[df['Sentence #'] == id].reset_index(drop=True)
    #entities = [(sent.index(tempdf['Word'][i]),sent.index(tempdf['Word'][i]) + len(sent.index(tempdf['Word'][i])) , tempdf['Tag'][i]) for i in range(len(tempdf)) if tempdf['Tag'][i] != "O"]
    sent = " ".join(tempdf['Word']).lower()
    sent=sent.replace(".", "").strip()
    #df_dum = tempdf[tempdf['Tag'] !='O'].reset_index(drop=False)
    entities_dict =dict()
    entities_dict['entities']=[]
    sent_backup = sent
    sent_length = 0
    for j in range(len(tempdf)):
        word = tempdf['Word'][j].lower()
        pattern = r'\b' + re.escape(word) + r'\b'
        match = re.search(pattern, sent)
        if match:
            str_ind = match.start()
            end_ind = match.end()
            entities_dict['entities'].append((str_ind+sent_length, end_ind+sent_length, tempdf['Tag'][j]))
            sent, sent_length = sent[end_ind:], len(sent[:end_ind])+sent_length
    return (sent_backup, entities_dict)

In [8]:
create=False
if create:
    input = [] 
    for i in tqdm(unique_sent_num):
        input.append(sentence_entities(i))
    #input = list(map(sentence_entities,unique_sent_num))
    # Open a file and use dump()
    with open('ner_input.pkl', 'wb') as file:
        # A new file will be created
        pickle.dump(input, file)  
else:
    file = open("ner_input_sep14.pkl",'rb')
    input = pickle.load(file)

In [9]:
input = input[0:100]
input[0]

('thousands of demonstrators have marched through london to protest the war in iraq and demand the withdrawal of british troops from that country',
 {'entities': [(0, 9, 'O'),
   (10, 12, 'O'),
   (13, 26, 'O'),
   (27, 31, 'O'),
   (32, 39, 'O'),
   (40, 47, 'O'),
   (48, 54, 'B-geo'),
   (55, 57, 'O'),
   (58, 65, 'O'),
   (66, 69, 'O'),
   (70, 73, 'O'),
   (74, 76, 'O'),
   (77, 81, 'B-geo'),
   (82, 85, 'O'),
   (86, 92, 'O'),
   (93, 96, 'O'),
   (97, 107, 'O'),
   (108, 110, 'O'),
   (111, 118, 'B-gpe'),
   (119, 125, 'O'),
   (126, 130, 'O'),
   (131, 135, 'O'),
   (136, 143, 'O')]})

In [12]:
# Define your labels and map them to numerical IDs
labels = ['O','B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim', 'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve', 'I-eve', 'I-nat']
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for label, i in label2id.items()}

In [13]:
train_data = []
wrong =[]
for i in tqdm(range(len(input))):
    text =  input[i][0]
    entities = []
    for j in input[i][1]['entities']:
        entities.append(j[2])
    sap_data = dict()
    text = text.split()
    text.insert(len(text),"[SEP]")
    text.insert(0,"[CLS]")
    sap_data['tokens'] = text
    entities.insert(len(entities),"O")
    entities.insert(0,"O")
    sap_data['labels'] = entities
    if len(entities)!=len(text):
        #print(i)
        wrong.append(i)
    train_data.append(sap_data)
len(wrong)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<?, ?it/s]


0

In [14]:
train_data[0]

{'tokens': ['[CLS]',
  'thousands',
  'of',
  'demonstrators',
  'have',
  'marched',
  'through',
  'london',
  'to',
  'protest',
  'the',
  'war',
  'in',
  'iraq',
  'and',
  'demand',
  'the',
  'withdrawal',
  'of',
  'british',
  'troops',
  'from',
  'that',
  'country',
  '[SEP]'],
 'labels': ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-gpe',
  'O',
  'O',
  'O',
  'O',
  'O']}

In [15]:
# Tokenize and encode your training data
# train_data = [
#     {"tokens": ["[CLS]", "John", "lives", "in", "New", "York", "[SEP]"],
#      "labels": ["O", "B-PER", "O", "O", "B-LOC", "I-LOC", "O"]}
#     # Add more examples
# ]

In [16]:
def encode_data(data):
    inputs = tokenizer(data["tokens"], is_split_into_words=True, return_offsets_mapping=True, padding="max_length", truncation=True)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    labels = [label2id[label] for label in data["labels"]]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [17]:
# Encode the training data
train_data = [encode_data(example) for example in train_data]

In [18]:
from transformers import Trainer, TrainingArguments
# Define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    num_train_epochs=1,
    output_dir="./ner_model",
    evaluation_strategy="epoch",
    save_total_limit=2,
    gradient_accumulation_steps=4
)

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


C:\Users\User\anaconda3\envs\nlp_2\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [19]:
# Create a data loader
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [20]:
import accelerate

In [21]:
# Access the model's configuration
config = model.config
# Check the number of labels supported by the model
num_labels = config.num_labels
print(f"Number of labels supported by the model: {num_labels}")

Number of labels supported by the model: 41


In [22]:
num_labels

41

In [23]:
len(train_data)

100

In [ ]:
# Create a trainer
from transformers import Trainer, TrainingArguments
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset = train_data
)
# Start training
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
results = trainer.evaluate()
print(results)

In [54]:
# Save the trained model to a directory
model.save_pretrained("NER_TRANSFORMER")

In [55]:
# Load the trained model from the saved directory
loaded_model = BertForTokenClassification.from_pretrained("NER_TRANSFORMER")

In [60]:
# Define a function for prediction
def predict_ner(sentence, model, tokenizer):
    # Tokenize and encode the input sentence
    inputs = tokenizer(sentence, truncation=True, padding=True, return_tensors="pt")

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted labels
    predicted_labels = torch.argmax(outputs.logits, dim=2).squeeze().tolist()
    predicted_labels = [model.config.id2label[label_id] for label_id in predicted_labels]

    return predicted_labels

In [58]:
import torch

def predict_ner(sentence, model, tokenizer):
    # Tokenize and encode the input sentence
    inputs = tokenizer(sentence, truncation=True, padding=True, return_tensors="pt")

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted labels
    predicted_labels = torch.argmax(outputs.logits, dim=2).squeeze().tolist()
    predicted_labels = [model.config.id2label[label_id] for label_id in predicted_labels]

    # Get the start and end indices of the entities
    entity_start = []
    entity_end = []
    current_entity_start = None
    for i, label in enumerate(predicted_labels):
        if label != "O":  # "O" indicates no entity
            if label.startswith("B-"):  # Beginning of an entity
                entity_start.append(i)
                current_entity_start = i
            elif label.startswith("I-") and current_entity_start is not None:
                entity_end.append(i)
            else:
                entity_start.append(i)  # Handle single-token entities
                entity_end.append(i)
                current_entity_start = None

    entities = []
    for start, end, label in zip(entity_start, entity_end, predicted_labels):
        entity_text = tokenizer.decode(inputs["input_ids"][0][start:end+1])
        # if entity_text in ['[CLS]','[SEP]']:
        #     add = 0
        #     start =0
        # else:
        #     start = start-1
        #     add = len(entity_text)-1
        entities.append({
            "start": start,
            "end": end,
            "label": label,
            "text": entity_text
        })

    return entities


In [61]:
# Example sentence for prediction
new_sentence = "John lives in Google New York and he works at Google"

# Make predictions
predicted_labels = predict_ner(new_sentence, loaded_model, tokenizer)
print(predicted_labels)

['LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0']


In [62]:
decoded_labels = [id2label[int(label.split("_")[1])] for label in predicted_labels]
decoded_labels

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']